# Creating a basic ingest of a NetCDF file

In [ ]:
### Sometimes a file might be in the netCDF file format but not conform to cfradial standards and can't
### be read with Py-ART. One way of working around this is to create a basic ingest, here is a hypothetically
### example, this file can be read with Py-ART, but lets act like it doesn't! :)

In [ ]:
import netCDF4
import pyart
import numpy as np

In [ ]:
# We will read the nc data with netCDF4.Dataset

In [ ]:
data = netCDF4.Dataset("test_radar.nc")

In [ ]:
# Lets get an idea of the shapes for rays and gates and the keys in this dataset.

In [ ]:
data.variables.keys()

In [ ]:
data["azimuth"][:].shape

In [ ]:
data["range"][:].shape

In [ ]:
# Make a empty radar with the dimensions of the dataset.

In [ ]:
radar = pyart.testing.make_empty_ppi_radar(667, 400, 1)

In [ ]:
# Start filling the radar attributes with variables in the dataset.

In [ ]:
radar.time["data"] = np.array(data["time"][:])

In [ ]:
data["longitude"][:]

In [ ]:
radar.latitude["data"] = np.array([data["latitude"][:]])

In [ ]:
radar.longitude["data"] = np.array([data["longitude"][:]])

In [ ]:
radar.longitude["data"]

In [ ]:
radar.range["data"] = np.array(data["range"][:])

In [ ]:
# Sometimes the dataset might just contain gate spacing, but if the gate spacing is uniform,
# there is a way around it, we see a gate spacing of 60 above.

In [ ]:
radar.range["data"] = np.linspace(0.0, 60.0 * (667 - 1), 667)

In [ ]:
# As you can see below we obtained the same range data. This isn't needed
# if the range data is present, but using gate spacing and ngates is another way around it.

In [ ]:
radar.range["data"]

In [ ]:
radar.fixed_angle["data"] = np.array(data["fixed_angle"])

In [ ]:
radar.sweep_number["data"] = np.array(data["sweep_number"])

In [ ]:
radar.sweep_start_ray_index["data"] = np.array(data["sweep_start_ray_index"])

In [ ]:
radar.sweep_end_ray_index["data"] = np.array(data["sweep_end_ray_index"])

In [ ]:
radar.altitude["data"] = np.array(data["altitude"])

In [ ]:
radar.azimuth["data"] = np.array(data["azimuth"])

In [ ]:
radar.sweep_mode["data"] = np.array(data["sweep_mode"])

In [ ]:
data["fixed_angle"][:]

In [ ]:
# If elevation doesn't exist, but fixed angle doesn't, you can do
# fixed angle multiplied by nrays

In [ ]:
radar.elevation["data"] = np.array(
    [data["fixed_angle"][:]] * len(data["azimuth"][:])
).squeeze()

In [ ]:
# With elevation and azimuth in the radar object, lets recalculate
# gate latitude, longitude and altitude,

In [ ]:
radar.init_gate_altitude()
radar.init_gate_longitude_latitude()

In [ ]:
radar.gate_longitude["data"]

In [ ]:
# Let's work on the field data, we will just do reflectivity for now, but any of the
# other fields can be done the same way and added as a key pair in the fields dict.

In [ ]:
from pyart.config import get_metadata

In [ ]:
ref_dict = get_metadata("reflecitivity_horizontal")

In [ ]:
ref_dict["data"] = np.array(data["reflectivity_horizontal"])

In [ ]:
ref_dict["data"]

In [ ]:
radar.fields = {"reflectivity_horizontal": ref_dict}

In [ ]:
# Now what does that data look like plotted with Py-ART, also confirm if it works.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
display = pyart.graph.RadarMapDisplay(radar)

In [ ]:
display.plot_ppi("reflectivity_horizontal")
plt.show()